<h2>Team Members</h2>
<ul>
<li>Sohad Hossam Eldin 1190019</li>
<li>Bassant Hisham Mohamed 1190018</li>
<li>Yasmin Hashem Niazy 4200013</li>
<li>Mary Ashraf 1190322</li>
</ul>

<h4>Imports</h4>

In [4]:
from imports import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sohad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<h4>preprocessing and feature extraction</h4>

In [5]:
def Tokenizing(dataset_sentences):
    harakat  = {
                    "": 0,     # No Diacritic
                    "َ": 1,     # Fatha
                    "ً": 2,     # Fathatah
                    "ُ": 3,     # Damma
                    "ٌ": 4,     # Dammatan
                    "ِ": 5,     # Kasra
                    "ٍ": 6,     # Kasratan
                    "ْ": 7,     # Sukun
                    "ّ": 8,     # Shaddah
                    "َّ": 9,     # Shaddah + Fatha
                    "ًّ": 10,    # Shaddah + Fathatah
                    "ُّ": 11,    # Shaddah + Damma
                    "ٌّ": 12,    # Shaddah + Dammatan
                    "ِّ": 13,    # Shaddah + Kasra
                    "ٍّ": 14,    # Shaddah + Kasratan
                            
                }
        
    arabic_alphabet = {
                            "ا": 1,
                            "ب": 2,
                            "ت": 3,
                            "ث": 4,
                            "ج": 5,
                            "ح": 6,
                            "خ": 7,
                            "د": 8,
                            "ذ": 9,
                            "ر": 10,
                            "ز": 11,
                            "س": 12,
                            "ش": 13,
                            "ص": 14,
                            "ض": 15,
                            "ط": 16,
                            "ظ": 17,
                            "ع": 18,
                            "غ": 19,
                            "ف": 20,
                            "ق": 21,
                            "ك": 22,
                            "ل": 23,
                            "م": 24,
                            "ن": 25,
                            "ه": 26,
                            "و": 27,
                            "ي": 28,
                            "آ": 29,
                            "إ": 30,
                            "ئ": 31,
                            "ء": 32,
                            "أ": 33,
                            "ؤ":34,
                            "ة":35,
                            "ى":36,
                        }

    x_train_letters = []
    y_train_letters = []
    x_train_words = []
    for sentence in dataset_sentences:
        sentence_wihout_diacratics = strip_tashkeel(sentence)
        tokens = word_tokenize(sentence, language="arabic", preserve_line=True)
        tokens_wihtout_diacratics = word_tokenize(sentence_wihout_diacratics, language="arabic", preserve_line=True)

        sentence_x = list()
        sentence_y = list()
        for word in tokens:
            text, inputs, diacritics =util.extract_haraqat(word)
            
            word_x = list()
            word_y = list()
            for i in range(len(inputs)):
                # for every char we see its rep in arabic_alphabet and get its 1 hot encoding vector
                word_x.append(arabic_alphabet[inputs[i]])
                word_y.append(harakat[diacritics[i]])
                # word_x.append(inputs[i])
                # word_y.append(diacritics[i])
            sentence_x.append(word_x)
            sentence_y.append(word_y)
        if(len(sentence_x) != 0):
            # 4d array len of sentences * len of words * len of chars * one hot encoding size
            x_train_letters.append(sentence_x)
            # 3d array len of sentences * len of words * number of diacritics --> represents the diacritics as indices check (harakat dict) 
            y_train_letters.append(sentence_y)

        if(len(tokens_wihtout_diacratics)) :
            x_train_words.append(tokens_wihtout_diacratics)


    print(y_train_letters[1])
    print(len(x_train_letters), len(x_train_letters[1]), len(x_train_letters[1][1]), x_train_letters[1][1][1])
    print(x_train_words[1])
    return x_train_letters,y_train_letters,x_train_words

dataset_txt = open(r"train.txt", "r", encoding='utf-8').read()
chars_to_remove = r"\(\s*[a-zA-Z0-9_-`]+\s*/\s*[a-zA-Z0-9_-`]+\s*\)|[a-zA-Z0-9_-]+|-|`|–|~|\u200f|'" 
dataset_cleaned = re.sub(chars_to_remove, "", dataset_txt)
dataset_sentences = re.split(r"\s*\.\s*|\n|\s*،\s*|\s*:\s*|\s*[()]\s*|\s*؛\s*|\s*؟\s*|\s*!\s*|\s*\"\s*|\s*\[\s*|\s*\]\s*|\s*{\s*|\s*}\s*|\s*\*\s*|\s*»\s*|\s*«\s*|\s*\\\s*|\s*/\s*|\s*;\s*|\s*,\s*",dataset_cleaned)

x_train_chars,y_train_chars,x_train_words = Tokenizing(dataset_sentences)

[[1, 7], [1, 1, 1], [0, 7, 1, 9, 3], [1, 1, 3], [0, 1, 7]]
278416 5 3 16
['أو', 'قطع', 'الأول', 'يده', 'إلخ']


In [ ]:
model = Word2Vec(sentences=x_train_words, vector_size=100, window=5, min_count=1, workers=4)

model.save("my_word2vec_model")

In [6]:
one_hot_enconding = np.eye(37) 
word2vec_model = Word2Vec.load("my_word2vec_model")
embedding_matrix = np.zeros((len(word2vec_model.wv.index_to_key) + 1, word2vec_model.vector_size))

word_index_x = {word: idx + 1 for idx, word in enumerate(word2vec_model.wv.index_to_key)}
for word, idx in word_index_x.items():
    embedding_matrix[idx] = word2vec_model.wv[word]

x_train_words_indices = [[word_index_x[word] for word in sentence] for sentence in x_train_words]

<h3>Dataset class<h3>

In [7]:
class ArabicDataset(Dataset):

    def __init__(self, x_train_words, x_train_chars, y_train_chars, one_hot_enconding):
        # the words dataset that will be given to the first model
        max_list_length = max(len(_) for _ in x_train_words)
        x_padded = [_ + [0] * (max_list_length - len(_)) for _ in x_train_words]
        self.tensor_x_words = torch.tensor(x_padded)

        x_padded_chars = [_ + [[]] * (max_list_length - len(_)) for _ in x_train_chars]
        #print(x_padded_chars[1])
        #print(len(x_padded_chars[1]),len(x_padded_chars[1][1]))

        # for sentence in x_train_chars:
        #     temp=list()
        #     for i in range(max_list_length - len(sentence)):
        #         temp.append([])
        #     sentence.extend(temp)

        max_char_length = max(len(word) for sentence in x_train_chars for word in sentence)
        for i in range(len(x_padded_chars)): #0->253
            for j in range(len(x_padded_chars[i])):
                temp=list()
                for k in range(max_char_length - len(x_padded_chars[i][j])):
                    temp.append(0)
                #print(temp)
                if(len(temp)!=0):
                    x_padded_chars[i][j].extend(temp)
        
        #print(len(x_padded_chars), len(x_padded_chars[2]), len(x_padded_chars[2][1]), x_padded_chars[2][1][1])
        self.tensor_x_chars = torch.tensor(x_padded_chars)

        max_char_length = max(len(word) for sentence in y_train_chars for word in sentence)
        #for sentence in y_train_chars: 
            #y_padded_chars.append([word + [0] * (max_char_length - len(word)) for word in sentence])
        y_padded_chars = [_ + [[]] * (max_list_length - len(_)) for _ in y_train_chars]    
        for i in range(len(y_padded_chars)): #0->253
            for j in range(len(y_padded_chars[i])):
                temp=list()
                for k in range(max_char_length - len(y_padded_chars[i][j])):
                    temp.append(0)
                #print(temp)
                if(len(temp)!=0):
                    y_padded_chars[i][j].extend(temp)
        self.tensor_y_chars = torch.tensor(y_padded_chars)

        #print(len(self.tensor_y_chars),len(self.tensor_y_chars[0]))
        #print(self.tensor_x_words[0], self.tensor_x_chars[0], self.tensor_y_chars[0])
        

    def __len__(self):

        return len(self.tensor_x_chars)

    # get a row at an index
    # The __getitem__ function loads and returns a sample from the dataset at the given index idx

    def __getitem__(self, idx):
        return self.tensor_x_chars[idx], self.tensor_y_chars[idx], self.tensor_x_words[idx]


<h4> Word2vec Feature Extraction </h4>

In [ ]:
dataset = ArabicDataset(x_train_words_indices, x_train_chars, y_train_chars, one_hot_enconding)
torch.save(dataset, 'ArabicDataset.pt')

In [ ]:
print(dataset.tensor_x_words[0])

In [8]:
dataset = torch.load('ArabicDataset.pt')

<h4> applying Bi-LSTM </h4>

In [10]:
class Bi_Lstm(nn.Module):

  def __init__(self,embedding_matrix : np.array, embedding_dim : tuple, one_hot_enconding:np.array,hot_encoding_vector_len : int=37 , num_chars : int =37 , hidden_size_layer_words : int=32, hidden_size_layer_chars : int =32, classes : int=15):
      super(Bi_Lstm, self).__init__()
      self.embedding_words=nn.Embedding(num_embeddings = embedding_dim[0], embedding_dim = embedding_dim[1], _weight = torch.tensor(embedding_matrix))
      self.lstm_layer_words = nn.LSTM(embedding_dim[1], hidden_size = hidden_size_layer_words, batch_first = True, bidirectional = True)
      self.embedding_char=nn.Embedding(num_embeddings = num_chars, embedding_dim = hot_encoding_vector_len, _weight = torch.tensor(one_hot_enconding))
      self.lstm_layer_char = nn.LSTM(hidden_size_layer_words * 2 + hot_encoding_vector_len, hidden_size=hidden_size_layer_chars, batch_first=True, bidirectional=True)
      self.linear = nn.Linear(2 * hidden_size_layer_chars, classes, bias=True)

  def forward(self,sentences,words):
      feature_vector = self.embedding_words(sentences) # sentences * words *253

      feature_vector = feature_vector.to(dtype=torch.float)
      lstm_output = self.lstm_layer_words(feature_vector) # sentences * words *200
      #print(lstm_output[0].shape)  
      # looping over lstm_output to remove unwanted feature vectors
      no_of_sentences, length_of_sentence, feature_vector_size = lstm_output[0].shape
      lstm_output_words= lstm_output[0]
      lstm_output_words_2d=lstm_output_words.reshape(-1,lstm_output_words.shape[2])
      #print(lstm_output_words_2d.shape)
      embedding_char_output_4d=self.embedding_char(words)  # sentences * words * chars * one hot encoding for each char
      #words * chars * one hot encoding for each char
      embedding_char_output_3d=embedding_char_output_4d.reshape(-1,embedding_char_output_4d.shape[2],embedding_char_output_4d.shape[3])
      #print(embedding_char_output_3d.shape)

      #print(lstm_output_words_2d[0]) # words*hot encoding vector
      embedding_char_output_3d_concatenated=torch.zeros(embedding_char_output_3d.shape[0],embedding_char_output_3d.shape[1],embedding_char_output_3d.shape[2]+lstm_output_words_2d.shape[1])
      for word in range(len(lstm_output_words_2d)):
            for chars in range(len(embedding_char_output_3d[word])):
               embedding_char_output_3d_concatenated[word][chars]  = torch.cat([embedding_char_output_3d[word][chars],lstm_output_words_2d[word]])
    # print(embedding_char_output_3d_concatenated.shape) 
      lstm_output_char = self.lstm_layer_char(embedding_char_output_3d_concatenated)
      final_output = self.linear(lstm_output_char[0])
      print(final_output.shape)
      return final_output


In [11]:
model=Bi_Lstm(embedding_matrix,(len(word2vec_model.wv.index_to_key) + 1, word2vec_model.vector_size),one_hot_enconding)

In [14]:
model.forward(dataset.tensor_x_words[:100],dataset.tensor_x_chars[:100])

torch.Size([25300, 200])
torch.Size([25300, 13, 15])


tensor([[[ 0.0358, -0.0114, -0.0013,  ...,  0.0158, -0.0482,  0.0400],
         [ 0.0134, -0.0198, -0.0141,  ...,  0.0198, -0.0561,  0.0339],
         [ 0.0237, -0.0176, -0.0095,  ...,  0.0187, -0.0664,  0.0403],
         ...,
         [ 0.0189, -0.0251,  0.0002,  ...,  0.0077, -0.0540,  0.0471],
         [ 0.0159, -0.0180, -0.0020,  ...,  0.0127, -0.0548,  0.0495],
         [ 0.0115, -0.0050, -0.0088,  ...,  0.0209, -0.0550,  0.0499]],

        [[ 0.0485,  0.0186,  0.0046,  ...,  0.0338, -0.0712,  0.0798],
         [ 0.0455,  0.0172,  0.0030,  ...,  0.0352, -0.0655,  0.0775],
         [ 0.0439,  0.0164,  0.0030,  ...,  0.0367, -0.0627,  0.0756],
         ...,
         [ 0.0368,  0.0164,  0.0045,  ...,  0.0422, -0.0587,  0.0736],
         [ 0.0338,  0.0184,  0.0019,  ...,  0.0432, -0.0567,  0.0715],
         [ 0.0287,  0.0223, -0.0043,  ...,  0.0438, -0.0523,  0.0658]],

        [[ 0.0490,  0.0225,  0.0094,  ...,  0.0420, -0.0713,  0.0835],
         [ 0.0468,  0.0207,  0.0087,  ...,  0

In [14]:
def train(model, train_dataset, batch_size=10, epochs=1, learning_rate=0.01):
      ############################## TODO: replace the Nones in the following code ##################################

    # (1) create the dataloader of the training set (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size)

    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss() 

    # (3) create the optimizer (Adam)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        n_samples=0

        for train_input_words, train_label, train_input_sentences in tqdm(train_dataloader):
            # print(train_input_words.shape, train_label.shape, train_input_sentences.shape)
            # (6) do the forward pass
            output = model.forward(train_input_sentences, train_input_words)
            
            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)
            # label: 100 * 253 * 13
            # output: 25300 word * 13 char * 15 
            # k2eny fket el array l words msh sentence of words
            # print(output.view(-1, output.size(2)).shape, train_label.view(-1).shape)

            batch_loss = criterion(output.view(-1, output.size(2)), train_label.view(-1))

            # (8) append the batch loss to the total_loss_train

            total_loss_train += batch_loss
            
            # (9) calculate the batch accuracy (just add the number of correct predictions)
            _,predicted=torch.max(output,2)  #512 * 104  for every word in every sentence we choose one tag form the seventen tag 
            acc=(predicted==train_label.view(-1, train_label.shape[2])).sum().item()
            n_samples += train_label.size(0)*train_label.size(1)*train_label.size(2)
            total_acc_train += acc
            print(100*total_acc_train/n_samples)

            # (10) zero your gradients
            optimizer.zero_grad()

            # (11) do the backward pass
            batch_loss.backward()
            nn.utils.clip_grad_norm(model.parameters(), max_norm=1)

            # (12) update the weights with your optimizer
            optimizer.step()
        
        # epoch loss
        epoch_loss = total_loss_train / len(train_dataset)

        # (13) calculate the accuracy
        epoch_acc =100*total_acc_train/n_samples

        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
            | Train Accuracy: {epoch_acc}\n')

  ##############################################################################################################

In [15]:
train(model, dataset)

  0%|          | 0/278416 [00:00<?, ?it/s]

torch.Size([1, 253, 13]) torch.Size([1, 253, 13]) torch.Size([1, 253])
torch.Size([253, 13, 15])
torch.Size([3289, 15]) torch.Size([3289])
0.0


C:\Users\sohad\AppData\Local\Temp\ipykernel_18956\3578052967.py:47: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  nn.utils.clip_grad_norm(model.parameters(), max_norm=1)
  0%|          | 1/278416 [00:01<137:17:07,  1.78s/it]

torch.Size([1, 253, 13]) torch.Size([1, 253, 13]) torch.Size([1, 253])
torch.Size([253, 13, 15])
torch.Size([3289, 15]) torch.Size([3289])
49.61994527211918


  0%|          | 2/278416 [00:03<136:09:42,  1.76s/it]

...

torch.Size([1, 253, 13]) torch.Size([1, 253, 13]) torch.Size([1, 253])
torch.Size([253, 13, 15])
torch.Size([3289, 15]) torch.Size([3289])
99.09760600297685


  0%|          | 306/278416 [10:39<183:32:56,  2.38s/it]

torch.Size([1, 253, 13]) torch.Size([1, 253, 13]) torch.Size([1, 253])
torch.Size([253, 13, 15])
torch.Size([3289, 15]) torch.Size([3289])
99.09925791528964


  0%|          | 306/278416 [10:41<162:00:05,  2.10s/it]


KeyboardInterrupt: 

## testing second model for char level encoding

In [ ]:
# looping over lstm_output to remove unwanted feature vectors
no_of_sentences, length_of_sentence, feature_vector_size = lstm_output.shape

lstm_output_without_padding = list()
for i in range(no_of_sentences):
    lstm_output_without_padding.extend(lstm_output[i, 1:len(dataset.x_train_words[i])-1, :])

# print(lstm_output_without_padding[0:6])


# x_train_letter_concatinated = [[np.concatenate(one_hot_enconding[letter_index], lstm_output_without_padding[i]) for letter_index in x_train_letter_indices[i]] for i in range(len(dataset.x_train_letters))]

x_train_letter_concatenated = []

for i in range(len(dataset.x_train_letters)):
    concatenated_arrays = []
    for letter in dataset.x_train_letters[i]:
        concatenated_arrays.append(np.concatenate([one_hot_enconding[dataset.arabic_alphabet[letter]], lstm_output_without_padding[i]]))
    x_train_letter_concatenated.extend(concatenated_arrays)


#[['ق', 'و', 'ل', 'ه'], ['أ', 'و'], ['ق', 'ط', 'ع'], ['ا', 'ل', 'أ', 'و', 'ل']]

In [ ]:
y_train_letter = [dataset.harakat[diacritic] for diacritic in dataset.y_train_letters]

## constructing model 2

In [ ]:
bi_lstm_model_char = Sequential()
first_bidirectional_layer = Bidirectional(LSTM(units=64, return_sequences=True))
bi_lstm_model_char.add(Dense(255, activation='softmax'))

bi_lstm_model_char.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

bi_lstm_model_char.fit(np.array(x_train_letter_concatenated), np.array(y_train_letter), epochs=10, batch_size=32)
score = bi_lstm_model_char.evaluate(np.array(x_train_letter_concatenated), np.array(y_train_letter))
print("Test accuracy:", score[1])

bi_lstm_model_char.save("char-model")

In [ ]:
print(x_train_padded.shape)

In [ ]:
word_input = Input(shape=(max_word_length,))
word_embedding = Embedding(input_dim=num_words, output_dim=embedding_dim)(word_input)
word_lstm = Bidirectional(LSTM(units=lstm_units, return_sequences=True))(word_embedding)